In [1]:
import tensorflow as tf

In [2]:
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

import pathlib

In [3]:
!tar -xf Data.tar.xz

In [11]:
data_dir = pathlib.Path('Data')
data_dir

PosixPath('Data')

In [12]:
!ls

Data  Data.tar.xz  sample_data


In [13]:
image_count = len(list(data_dir.glob('*/*.jpeg')))
image_count

1000

In [14]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*')])
CLASS_NAMES

array(['turkey', 'chicken'], dtype='<U7')

In [15]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [16]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

In [17]:
list_ds = tf.data.Dataset.list_files([str(data_dir/'*/*.jpeg')])

In [18]:
for f in list_ds.take(5):
    print(f.numpy())

b'Data/chicken/images125.jpeg'
b'Data/chicken/images179.jpeg'
b'Data/turkey/images190.jpeg'
b'Data/turkey/images334.jpeg'
b'Data/turkey/images302.jpeg'


In [19]:
def get_label(file_path):
    parts = tf.strings.split(file_path, sep= os.path.sep)
    return tf.cast((parts[-2] == CLASS_NAMES), dtype='uint8')

In [20]:
def decode_img(img):
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

In [21]:
def process_path(file_path):
    label = get_label(file_path)
    img = decode_img(tf.io.read_file(file_path))
    return img, label

In [22]:
labeled_ds = list_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [23]:
for item in labeled_ds.take(1):
    print(item[0].shape)

(224, 224, 3)


In [51]:
ds = labeled_ds.shuffle(buffer_size=1000, seed=42)

In [52]:
import tensorflow_datasets as tfds

In [44]:
dds = tfds.as_numpy(ds)

In [47]:
xx,yy =[],[]
for i,j in dds:
  xx.append(i)
  yy.append(j)

In [53]:
x = np.array(xx)
y = np.array(yy)

In [74]:
tf.keras.backend.clear_session()

In [75]:
input_ = tf.keras.layers.Input(shape=[224,224,3])
conv1 = tf.keras.layers.Conv2D(64, 5, activation= 'relu', padding='same')(input_)
pool1 = tf.keras.layers.MaxPool2D(2)(conv1)
conv2 = tf.keras.layers.Conv2D(128, 5, activation= 'relu', padding='same')(pool1)
pool2 = tf.keras.layers.MaxPool2D(2)(conv2)
conv3 = tf.keras.layers.Conv2D(256, 3, activation= 'relu', padding='same')(pool2)
pool3 = tf.keras.layers.MaxPool2D(2)(conv3)
conv4 = tf.keras.layers.Conv2D(512, 3, activation= 'relu', padding='same')(pool3)
pool4 = tf.keras.layers.MaxPool2D(2)(conv4)
conv5 = tf.keras.layers.Conv2D(1024, 3, activation= 'relu', padding='same')(pool4)
pool5 = tf.keras.layers.MaxPool2D(2)(conv5)
conv6 = tf.keras.layers.Conv2D(2048, 3, activation= 'relu', padding='same')(pool5)
pool6 = tf.keras.layers.MaxPool2D(2)(conv6)
flat = tf.keras.layers.Flatten()(pool6)
dense1 = tf.keras.layers.Dense(1024, activation= 'relu')(flat)
drop1 = tf.keras.layers.Dropout(0.5)(dense1)
dense2 = tf.keras.layers.Dense(256, activation= 'relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.5)(dense2)
dense3 = tf.keras.layers.Dense(64, activation= 'relu')(drop2)
dense4 = tf.keras.layers.Dense(16, activation= 'relu')(dense3)
output_ = tf.keras.layers.Dense(2, activation= 'softmax')(dense4)

In [76]:
model = tf.keras.Model(inputs=[input_], outputs=[output_])

In [77]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      4864      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 128)     204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 256)       0     

In [78]:
model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=0.0001), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [79]:
history = model.fit(x,y, epochs=100,validation_split=0.1, callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

Epoch 1/100
29/29 [==============================] - 3s 113ms/step - loss: 0.6955 - accuracy: 0.5028 - val_loss: 0.6925 - val_accuracy: 0.4900
Epoch 2/100
29/29 [==============================] - 3s 106ms/step - loss: 0.6934 - accuracy: 0.4983 - val_loss: 0.6916 - val_accuracy: 0.4900
Epoch 3/100
29/29 [==============================] - 3s 104ms/step - loss: 0.6934 - accuracy: 0.4894 - val_loss: 0.6912 - val_accuracy: 0.4900
Epoch 4/100
29/29 [==============================] - 3s 105ms/step - loss: 0.6856 - accuracy: 0.5250 - val_loss: 0.6779 - val_accuracy: 0.4900
Epoch 5/100
29/29 [==============================] - 3s 106ms/step - loss: 0.6624 - accuracy: 0.5929 - val_loss: 0.6226 - val_accuracy: 0.7200
Epoch 6/100
29/29 [==============================] - 3s 104ms/step - loss: 0.6089 - accuracy: 0.7097 - val_loss: 0.6023 - val_accuracy: 0.6600
Epoch 7/100
29/29 [==============================] - 3s 106ms/step - loss: 0.5867 - accuracy: 0.7063 - val_loss: 0.5427 - val_accuracy: 0.7100

In [81]:
model.save('withoutResNet.h5')